In [2]:
# Mechanics Roster Optimization
# Import required libraries
import pandas as pd
from ortools.linear_solver import pywraplp

# Load data files
print("Loading data files...")

# Load mechanic skills dataset
mechanic_skills_df = pd.read_excel('mechanic_skills_dataset.xlsx')
print("\nMechanic Skills Dataset:")
print(mechanic_skills_df.head())
print(f"\nShape: {mechanic_skills_df.shape}")

# Extract mechanic IDs
mechanics = sorted(mechanic_skills_df['mechanic_id'].unique().tolist())
print(f"\nMechanics: {mechanics}")
print(f"Number of mechanics: {len(mechanics)}")

# Load base aircraft schedule
base_schedule_df = pd.read_excel('base_aircraft_schedule.xlsx')
print("\n\nBase Aircraft Schedule:")
print(base_schedule_df.head())
print(f"\nShape: {base_schedule_df.shape}")

# Extract unique base IDs
bases = sorted(base_schedule_df['base_id'].unique().tolist())
print(f"\nBases: {bases}")
print(f"Number of bases: {len(bases)}")

# Extract periods (groups)
periods = sorted(base_schedule_df['period'].unique().tolist())
print(f"\nPeriods (Groups): {periods}")

# Extract shifts
shifts = sorted(base_schedule_df['shift'].unique().tolist())
print(f"Shifts: {shifts} (1=day, 2=night)")

# Load cost matrix
cost_matrix_df = pd.read_excel('cost_matrix.xlsx')
print("\n\nCost Matrix:")
print(cost_matrix_df.head())
print(f"\nShape: {cost_matrix_df.shape}")

# Map cost columns A, B, C to base IDs
# Assuming A→1, B→2, C→3 (verify from base_schedule)
base_column_mapping = {'A': 1, 'B': 2, 'C': 3}
print(f"\nBase column mapping: {base_column_mapping}")

# Create cost dictionary: cost[mechanic_id, base_id]
cost_dict = {}
for _, row in cost_matrix_df.iterrows():
    mechanic_id = int(row['id'])
    for col, base_id in base_column_mapping.items():
        if col in cost_matrix_df.columns:
            cost_dict[(mechanic_id, base_id)] = float(row[col])

print(f"\nCost dictionary created with {len(cost_dict)} entries")
print("Sample costs:")
for i, (key, val) in enumerate(list(cost_dict.items())[:5]):
    print(f"  Mechanic {key[0]} to Base {key[1]}: {val}")

# Verify data consistency
print("\n\nData Validation:")
print(f"Mechanics in skills dataset: {len(mechanics)}")
print(f"Mechanics in cost matrix: {len(cost_matrix_df)}")
print(f"Bases in schedule: {bases}")
print(f"Bases in cost matrix: {list(base_column_mapping.values())}")

Loading data files...

Mechanic Skills Dataset:
   mechanic_id  aw139_af  aw139_r  aw139_av  h175_af  h175_r  h175_av  \
0            1         1        1         0        0       0        0   
1            2         0        0         1        0       0        1   
2            3         0        0         1        1       1        1   
3            4         1        1         0        0       0        1   
4            5         0        0         1        0       0        0   

   sk92_af  sk92_r  sk92_av  
0        1       1        0  
1        0       0        1  
2        0       0        0  
3        0       0        0  
4        1       1        1  

Shape: (49, 10)

Mechanics: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
Number of mechanics: 49


Base Aircraft Schedule:
   base_id  aw139  h175  sk92  period  shift
0        1      1    

In [ ]:
# Set up OR-Tools solver
print("Setting up OR-Tools solver...")

# Create solver instance (using SCIP for mixed integer programming)
solver = pywraplp.Solver.CreateSolver('SCIP')

if not solver:
    print("SCIP solver not available, using CBC instead")
    solver = pywraplp.Solver.CreateSolver('CBC')

print(f"Solver: {solver.SolverVersion()}")

# Create decision variables: x[mechanic, base, group, shift]
# x[m, b, g, s] = 1 if mechanic m is assigned to base b in group g with shift s
# x[m, b, g, s] = 0 otherwise

print("\nCreating decision variables...")
x = {}

for m in mechanics:
    for b in bases:
        for g in periods:  # periods are the groups
            for s in shifts:
                var_name = f'x_m{m}_b{b}_g{g}_s{s}'
                x[(m, b, g, s)] = solver.IntVar(0, 1, var_name)

print(f"Created {len(x)} binary decision variables")
print(f"Sample variable: {x[(mechanics[0], bases[0], periods[0], shifts[0])]}")

Setting up OR-Tools solver...
Solver: SCIP 10.0.0 [LP solver: SoPlex 8.0.0]


AttributeError: type object 'Solver' has no attribute 'SupportedSolvers'

In [ ]:
# Constraint 1: Each mechanic can work in at most one shift, one group, and one base
# Σ(b∈Bases) Σ(g∈Groups) Σ(s∈Shifts) x[m, b, g, s] ≤ 1  ∀m ∈ Mechanics

print("Adding first constraint: Each mechanic ≤ 1 assignment...")

constraint_count = 0
for m in mechanics:
    constraint = solver.Constraint(0, 1, f'mechanic_{m}_single_assignment')
    for b in bases:
        for g in periods:
            for s in shifts:
                constraint.SetCoefficient(x[(m, b, g, s)], 1)
    constraint_count += 1

print(f"Added {constraint_count} constraints (one per mechanic)")
print(f"Total constraints so far: {solver.NumConstraints()}")

In [ ]:
# Constraint 2: Skill Coverage Constraint
# For each (base, period, shift) combination, the pool of assigned mechanics must 
# collectively have all required skills for all aircraft types present at that base
# For each aircraft type a present at (b, p, s):
#   Σ(m ∈ Mechanics with skill a_af) x[m, b, p, s] ≥ 1
#   Σ(m ∈ Mechanics with skill a_r) x[m, b, p, s] ≥ 1
#   Σ(m ∈ Mechanics with skill a_av) x[m, b, p, s] ≥ 1

print("Adding skill coverage constraints...")

# Aircraft types and their corresponding skill columns
aircraft_types = ['aw139', 'h175', 'sk92']
skill_types = ['_af', '_r', '_av']

# Create a mapping of mechanics to their skills for quick lookup
mechanic_skills = {}
for _, row in mechanic_skills_df.iterrows():
    m = int(row['mechanic_id'])
    mechanic_skills[m] = {}
    for aircraft in aircraft_types:
        for skill in skill_types:
            col_name = f"{aircraft}{skill}"
            if col_name in mechanic_skills_df.columns:
                mechanic_skills[m][col_name] = int(row[col_name])

constraint_count = 0

# Iterate through each row in base_aircraft_schedule
for _, row in base_schedule_df.iterrows():
    base_id = int(row['base_id'])
    period = int(row['period'])
    shift = int(row['shift'])
    
    # Check which aircraft types are present (count > 0)
    for aircraft in aircraft_types:
        if aircraft in base_schedule_df.columns:
            aircraft_count = row[aircraft]
            if aircraft_count > 0:  # Aircraft type is present
                # For each skill type (_af, _r, _av), ensure at least one mechanic with that skill is assigned
                for skill in skill_types:
                    skill_name = f"{aircraft}{skill}"
                    constraint = solver.Constraint(1, solver.infinity(), 
                                                  f'skill_{skill_name}_base{base_id}_period{period}_shift{shift}')
                    
                    # Find all mechanics with this skill
                    for m in mechanics:
                        if mechanic_skills[m].get(skill_name, 0) == 1:
                            constraint.SetCoefficient(x[(m, base_id, period, shift)], 1)
                    
                    constraint_count += 1

print(f"Added {constraint_count} skill coverage constraints")
print(f"Total constraints: {solver.NumConstraints()}")

In [ ]:
# Objective Function: Minimize total cost of moving mechanics to bases
# Minimize: Σ(m∈Mechanics) Σ(b∈Bases) Σ(g∈Groups) Σ(s∈Shifts) cost[m, b] * x[m, b, g, s]

print("Creating objective function...")

objective = solver.Objective()

for m in mechanics:
    for b in bases:
        # Get cost for this mechanic-base pair
        cost = cost_dict.get((m, b), 0)
        for g in periods:
            for s in shifts:
                objective.SetCoefficient(x[(m, b, g, s)], cost)

objective.SetMinimization()

print("Objective function created: Minimize total movement cost")
print(f"Number of variables in objective: {len(x)}")
print(f"Model summary:")
print(f"  Variables: {solver.NumVariables()}")
print(f"  Constraints: {solver.NumConstraints()}")

In [ ]:
# Solve the optimization problem
print("Solving the optimization problem...")
print("This may take a few moments...\n")

status = solver.Solve()

# Check solution status
if status == pywraplp.Solver.OPTIMAL:
    print("✓ Optimal solution found!\n")
    print(f"Optimal total cost: {objective.Value():.2f}\n")
    
    # Display assignments
    print("Mechanic Assignments:")
    print("-" * 80)
    print(f"{'Mechanic':<10} {'Base':<8} {'Group':<8} {'Shift':<8} {'Cost':<10}")
    print("-" * 80)
    
    assignments = []
    total_cost = 0
    for m in mechanics:
        for b in bases:
            for g in periods:
                for s in shifts:
                    if x[(m, b, g, s)].solution_value() > 0.5:  # Binary variable is 1
                        cost = cost_dict.get((m, b), 0)
                        total_cost += cost
                        shift_name = "Day" if s == 1 else "Night"
                        print(f"{m:<10} {b:<8} {g:<8} {shift_name:<8} {cost:<10.2f}")
                        assignments.append({
                            'mechanic_id': m,
                            'base_id': b,
                            'group': g,
                            'shift': s,
                            'shift_name': shift_name,
                            'cost': cost
                        })
    
    print("-" * 80)
    print(f"\nTotal assignments: {len(assignments)}")
    print(f"Total cost: {total_cost:.2f}")
    print(f"\nUnassigned mechanics: {len(mechanics) - len(assignments)}")
    
elif status == pywraplp.Solver.FEASIBLE:
    print("✓ Feasible solution found (may not be optimal)\n")
    print(f"Total cost: {objective.Value():.2f}\n")
elif status == pywraplp.Solver.INFEASIBLE:
    print("✗ Problem is infeasible - no solution exists that satisfies all constraints")
elif status == pywraplp.Solver.UNBOUNDED:
    print("✗ Problem is unbounded")
else:
    print(f"✗ Solver status: {status}")